<img src="logo.png" width="200">

<b>Análise de acessos ao domínio da empresa:</b><br><br>

<font color='darkblue'>
<b><br>Executar todas as células. Resultado aparece no final da página.<br></b><br>
</font>

In [ ]:
#Carregando as bibliotecas necessárias, primeira parte
# coding: utf-8
import pandas as pd
import re
from unicodedata import normalize
from IPython.display import display, HTML
import matplotlib
import matplotlib.pyplot as plt
plt.style.available
import xlwings as xw
import os
from subprocess import Popen
import os,sys
from functools import reduce

In [ ]:
#Carregando as bibliotecas necessárias e configurando notebook, segunda parte
%matplotlib inline

In [ ]:
#Carregando as bibliotecas necessárias e configurando notebook, segunda parte
import venn
import matplotlib
#matplotlib.use('Agg')
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles

In [ ]:
#Carregando as bibliotecas necessárias e configurando notebook, segunda parte
import plotly.graph_objects as go
!jupyter nbextension enable --py widgetsnbextension

from chart_studio.plotly import plot, iplot as py
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [ ]:
#Carregando as bibliotecas necessárias e configurando notebook, segunda parte
from IPython.core.display import HTML
HTML("""
    <style>
    .output_png {
        display: table-cell;
        text-align: center;
        vertical-align: middle;
    }
    </style>
    """)

In [ ]:
#Criando dicionário para substituir letras com acentos
dictionary_acentos = {'á':'a','é':'e','í':'i','ó':'o','ú':'u',
              'â':'a','ê':'e','î':'i','ô':'o','û':'u',
              'ã':'a','õ':'o',
              'à':'a','è':'e','ì':'i','ò':'o','ù':'u',
              'ç':'c'}

In [ ]:
#Indicando arquivos e diretórios para obtenção dos dados de usuários e grupos da rede
diretorio_bases = str('') ####colocar diretorio
base_grupos = str('arquivo.csv') ###colocar nome do arquivo
base_usuarios_levantamento = str('arquivo.txt')###colocar nome do arquivo
diretorio_resultado_consulta = str('') ###colocar nome do diretorio
base_rh = str('arquivo_com_dados_dos_funcionarios_conforme_rh.xls')###colocar nome do arquivo
arquivo_grupos = os.path.join(diretorio_bases, base_grupos)
arquivo_usuarios_levantamento = os.path.join(diretorio_bases, base_usuarios_levantamento)
detalhe_usuarios = os.listdir(diretorio_resultado_consulta)
relacao_rh = os.path.join(diretorio_bases,base_rh)

In [ ]:
#Função para criar df com os grupos existentes na base consultada
def base_grupos():
    with open(arquivo_grupos) as g:
        tmp_grupos_rede = [line.rstrip('\n') for line in g]
    grupos_rede = pd.DataFrame(tmp_grupos_rede[6:-2], columns=[0])
    grupos_rede.reset_index(drop=True).rename(columns={0:'Grupos'})
    return grupos_rede

In [ ]:
#Função para criar dfs diversos com os usuarios existentes na base consultada, arquivo com detalhes sobre grupos
def base_detalhe_usuarios():
    nomes_dos_arquivos=[]
    arquivos = {}
    login = []
    status = []
    comentario = []
    ultimo_logon = []
    grupos = []
    for nome in detalhe_usuarios:
        if nome != "tmp_users.txt":
            nomes_dos_arquivos.append(nome)
            linhanum = 0
            login_padrao = re.compile(r"^Nome de u.*")
            status_padrao = re.compile(r"^Conta ativa.*")
            comentario_padrao = re.compile(r"Coment.rio+[\s]+[\s].*")
            ultimo_logon_padrao = re.compile(r".*o logon.*")
            grupos_pri_padrao = re.compile(r".*Grupo Global.*")
            grupos_seg_padrao = re.compile(r'^\W+[\w]')#(r'^([\w]+)\W+[\w]')
            endereco = os.path.join(diretorio_resultado_consulta, nome)
            with open (endereco, 'r', encoding='iso-8859-1') as u:
               for linha_arquivo in u:
                    linhanum += 1
                    if login_padrao.search(linha_arquivo) != None:
                       login.append(nome + ',' + linha_arquivo.rstrip())
                    if status_padrao.search(linha_arquivo) != None:
                        status.append((nome + ',' + linha_arquivo.rstrip()))
                    if comentario_padrao.search(linha_arquivo) != None:
                        comentario.append((nome + ',' + linha_arquivo.rstrip()))               
                    if ultimo_logon_padrao.search(linha_arquivo) != None:
                        ultimo_logon.append((nome + ',' + linha_arquivo.rstrip()))
                    if grupos_pri_padrao.search(linha_arquivo) != None:
                        grupos.append((nome + ',' + linha_arquivo.rstrip()))
                    if grupos_seg_padrao.search(linha_arquivo) != None:
                        grupos.append((nome + ',Associações de Grupo Global' + linha_arquivo.rstrip()))    
        else:
            None
    #Ajustando encoding manualmente
    login = [item.replace("\x87", "ç").replace("\xa0", "á").replace("Æ", "ã").replace("¡", "í").replace("él","ú").replace("ä","õ").replace("\x82", "é") for item in login]
    status = [item.replace("\x87", "ç").replace("\xa0", "á").replace("Æ", "ã").replace("¡", "í").replace("él","ú").replace("ä","õ").replace("\x82", "é") for item in status]
    comentario = [item.replace("\x87", "ç").replace("\xa0", "á").replace("Æ", "ã").replace("¡", "í").replace("él","ú").replace("ä","õ").replace("\x82", "é") for item in comentario]
    ultimo_logon = [item.replace("\x87", "ç").replace("\xa0", "á").replace("Æ", "ã").replace("¡", "í").replace("él","ú").replace("ä","õ").replace("\x82", "é") for item in ultimo_logon]
    grupos = [item.replace("\x87", "ç").replace("\xa0", "á").replace("Æ", "ã").replace("¡", "í").replace("él","ú").replace("ä","õ").replace("\x82", "é") for item in grupos]

    login = (pd.DataFrame([i.split(',Nome de usuário') for i in login]))
    login.rename(columns={0:'Origem',1:'Login'}, inplace=True)
    status = (pd.DataFrame([i.split(',Conta ativa') for i in status]))
    status.rename(columns={0:'Origem',1:'Status_Ativo'}, inplace=True)
    comentario = (pd.DataFrame([i.split(',Comentário') for i in comentario]))
    comentario.rename(columns={0:'Origem',1:'Comentario'}, inplace=True)
    ultimo_logon = (pd.DataFrame([i.split(',útimo logon') for i in ultimo_logon]))
    ultimo_logon.rename(columns={0:'Origem',1:'Ultimo_Logon'}, inplace=True)
    grupos = (pd.DataFrame([i.split(',Associações de Grupo Global') for i in grupos]))
    grupos.rename(columns={0:'Origem',1:'Grupos'}, inplace=True)

    return (login, status, comentario, ultimo_logon, grupos)

In [ ]:
#Função para criar df unico com os usuarios ativos, tanto do detalhe quanto do levantamento geral
def base_usuarios_rede_ativos():
    login, status, comentario, ultimo_logon, grupos = base_detalhe_usuarios()
    data_frames = [login, status, comentario, ultimo_logon, grupos]
    usuarios_rede_detalhe = reduce(lambda  left,right: pd.merge(left,right,on=['Origem'],how='outer'), data_frames).fillna('nulo')
    usuarios_rede_detalhe['Tipo'] = 'Rede'
    usuarios_rede_detalhe_ativos = usuarios_rede_detalhe[usuarios_rede_detalhe['Status_Ativo'].str.contains('|'.join('Sim'), na = False)]
    usuarios_rede_detalhe_ativos.replace('(^\s+|\s+$)', '', regex=True, inplace=True)

    usuarios_rede_levantamento = pd.read_csv(arquivo_usuarios_levantamento)

    usuarios_rede_ativos = pd.merge(usuarios_rede_detalhe_ativos,usuarios_rede_levantamento, left_on='Login', right_on='sAMAccountName', how='left').fillna('nulo')
    usuarios_rede_ativos['mail'] = usuarios_rede_ativos['mail'].str.lower()
    usuarios_rede_ativos['mail'].replace(dictionary_acentos, regex=True, inplace=True)
    usuarios_rede_ativos['department'] = usuarios_rede_ativos['department'].str.lower()
    usuarios_rede_ativos['department'].replace(dictionary_acentos, regex=True, inplace=True)
    usuarios_rede_ativos['mail'] = usuarios_rede_ativos['mail'].str.strip()

    return usuarios_rede_ativos

In [ ]:
#Função para criar df com dados de funcionários ativos conforme relação do RH
def base_funcionarios_ativos_rh():
    sheet = xw.Book(relacao_rh).sheets['Ativos_jun'].used_range
    relacao_ativos_rh = pd.DataFrame(xw.Book(relacao_rh).sheets['Ativos_jun'].used_range.value)
    new_header = relacao_ativos_rh.iloc[0]
    relacao_ativos_rh = relacao_ativos_rh[1:]
    relacao_ativos_rh.columns = new_header
    relacao_ativos_rh.reset_index(drop=True, inplace=True)
    relacao_ativos_rh['mail'] = relacao_ativos_rh['mail'].str.lower()
    relacao_ativos_rh['mail'].replace(dictionary_acentos, regex=True, inplace=True)
    relacao_ativos_rh['Centro_Resultado_Nome'] = relacao_ativos_rh['Centro_Resultado_Nome'].str.lower()
    relacao_ativos_rh['Centro_Resultado_Nome'].replace(dictionary_acentos, regex=True, inplace=True)
    relacao_ativos_rh['Empresa_Razão_Social'] = relacao_ativos_rh['Empresa_Razão_Social'].str.lower()
    relacao_ativos_rh['Empresa_Razão_Social'].replace(dictionary_acentos, regex=True, inplace=True)
    relacao_ativos_rh =relacao_ativos_rh.drop(['CPF'], axis=1)
    relacao_ativos_rh['mail'] = relacao_ativos_rh['mail'].str.strip()
    relacao_ativos_rh['Nome'] = relacao_ativos_rh['Nome'].str.strip()
    #quebra_nome_lista = (pd.DataFrame([ x.split() for x in relacao_ativos_rh['mail'].tolist()]))
    #relacao_ativos_rh['mail'] = quebra_nome_lista[0]
    #relacao_ativos_rh['mail'].astype(str)
    return relacao_ativos_rh

In [ ]:
relacao_ativos_rh = base_funcionarios_ativos_rh()
usuarios_rede_ativos = base_usuarios_rede_ativos()

In [ ]:
#Gerar comparativo entre bases para arquivos de localizados e não localizados
def comparativo():
    #relacao_ativos_rh = base_funcionarios_ativos_rh()
    #usuarios_rede_ativos = base_usuarios_rede_ativos()
    comparacao = pd.merge(usuarios_rede_ativos, relacao_ativos_rh, left_on='mail', right_on='mail', how='outer', indicator=True)
    usuarios_localizados = comparacao[comparacao._merge.isin(['both'])]
    #display(HTML(usuarios_localizados[:3].to_html(index=False)))
    usuarios_localizados.to_excel(os.path.join(diretorio_bases, 'usuarios_localizados.xlsx'))
    usuarios_nao_localizados = comparacao[~comparacao._merge.isin(['both'])]
    #display(HTML(usuarios_nao_localizados[:3].to_html(index=False)))
    usuarios_nao_localizados.to_excel(os.path.join(diretorio_bases, 'usuarios_nao_localizados.xlsx'))
    return usuarios_localizados, usuarios_nao_localizados, comparacao

In [ ]:
usuarios_localizados, usuarios_nao_localizados, comparacao = comparativo()

In [ ]:
#Diagrama de venn
def diagrama_venn():
    set_rede = usuarios_rede_ativos[['mail', 'Comentario']]
    set_rh =  relacao_ativos_rh[['mail','Empresa_Razão_Social']]

    lst_rede = set_rede['mail'].tolist()
    lst_rh = set_rh['mail'].tolist()
    
    tpl_rh = sorted(set(map(tuple, lst_rh)), reverse=True)
    tpl_rede = sorted(set(map(tuple, lst_rede)), reverse=True)

    v = venn2([set(lst_rh), set(lst_rede)], set_labels = ('RH', 'Rede'))
    c = venn2_circles([set(lst_rh), set(lst_rede)],linestyle='dashed')
    c[1].set_lw(0.5)
    c[0].set_lw(1.5)
    c[0].set_ls('dotted')
    for circle, color in zip(c, ['#DD5F13', '#45548D']):
        circle.set_lw(2.0)
        circle.set_ls('dotted')
        circle.set_alpha(0.5)
        circle.set_color(color)
    csfont = {'fontname':'Arial', 'size':16, 'color': '#2A3F5F'}
    plt.title('Comparação entre funcionários e logins ativos no domínio consultado',**csfont)
    return plt.show()

In [ ]:
#Gráficos agrupando não localizados por categorias
def categorias_n_loc():
    

    nao_localizados_rede = usuarios_nao_localizados.loc[usuarios_nao_localizados['_merge'] == 'left_only'][['mail','Comentario']]
    nao_localizados_rede = nao_localizados_rede.groupby('Comentario',as_index=True).agg(['nunique'])
    nao_localizados_rede = nao_localizados_rede.reset_index()
    nao_localizados_rede = nao_localizados_rede.droplevel(1, axis=1)
    nao_localizados_rede.rename(columns={'Comentario':'Categoria', 'mail':'Qtd'}, inplace=True)
    nao_localizados_rede['Categoria'] = nao_localizados_rede['Categoria'].replace('','Sem comentários para o login')
    nao_localizados_rede = nao_localizados_rede.sort_values(by='Qtd', ascending=False).reset_index(drop=True)
     
    nao_localizados_rh = pd.DataFrame(usuarios_nao_localizados.loc[usuarios_nao_localizados['_merge'] == 'right_only'][['mail','Empresa_Razão_Social']].groupby(['Empresa_Razão_Social'],as_index=True).size()).sort_values(by=0, ascending=False).reset_index()
    nao_localizados_rh.rename(columns={0:'Qtd', 'Empresa_Razão_Social':'Categoria'}, inplace=True)
    
    trace1 = go.Bar(
                x = nao_localizados_rh.Categoria,
                y = nao_localizados_rh.Qtd,
                name = "Qtd",
                marker = dict(color = '#008295',
                             line=dict(color='rgb(0,0,0)',width=1),
                             opacity=0.6), 
                offsetgroup=0,
                text=nao_localizados_rh.Qtd,
                textposition='outside')
    data1 = [trace1]
    layout1 = go.Layout(barmode = "group",
        xaxis = go.XAxis(showticklabels=True, automargin=False, tickangle = 15,
        title_standoff = 5),
        yaxis = go.XAxis(showticklabels=False),
        margin=dict(l=300, r=70,t=70,b=120),
        title = "Funcionários ativos com login não localizado no domínio consultado")
    fig1 = go.Figure(data = data1, layout = layout1)
    fig1.update_layout(width=900, height=600, title={'font_family':"Arial",'y':0.95, 'x':0.5,'xanchor': 'center','yanchor': 'top'})

    trace2 = go.Bar(
                y = nao_localizados_rede.Categoria,
                x = nao_localizados_rede.Qtd,
    
                name = "Qtd",
                marker = dict(color = '#008295',
                             line=dict(color='rgb(0,0,0)',width=1),
                             opacity=0.6), 
                offsetgroup=0,
                text=nao_localizados_rede.Qtd,
                textposition='outside', orientation='h')
    data2 = [trace2]
    layout2 = go.Layout(barmode = "group",
        yaxis = go.XAxis(showticklabels=True, automargin=False, tickangle = 0, tickfont = {"size":9},
        title_standoff = 5),
        xaxis = go.XAxis(showticklabels=False),
        margin=dict(l=300, r=70,t=70,b=70),
        title='Logins com email não localizado no controle mais atual do RH')
    fig2 = go.Figure(data = data2, layout = layout2)
    fig2.update_layout(width=900, height=1500, title={'font_family':"Arial",'y':0.98, 'x':0.5,'xanchor': 'center','yanchor': 'top'})
    fig2.update_layout( yaxis={'categoryorder':'total ascending'})

    return iplot(fig1), iplot(fig2)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
!jupyter nbextension enable --py widgetsnbextension

from floweaver import *
from ipywidgets import *

In [ ]:
#Criando função que traz lista de valores
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

In [ ]:
#Criando botões
def dropdowns():
    global dropdown_agrupamento,dropdown_grupos_rede 
    global output_resultado
    #plot_output_agrupamento, plot_output_grupos_rede

    dropdown_agrupamento = widgets.Dropdown(options = ['Centro_Resultado_Nome', 'Cargo'], description='Selecione:')
    dropdown_grupos_rede = widgets.Dropdown(options = unique_sorted_values(usuarios_localizados.Grupos.astype('str')))

    output_resultado = widgets.Output()

    
    return;

In [ ]:
#Transformando botões em filtros
def common_filtering(agrupamento, grupos_rede):
    global common_filter, b
    
    output_resultado.clear_output()
    
    if (agrupamento == 'Centro_Resultado_Nome'):
        a = usuarios_localizados.loc[usuarios_localizados['Grupos'] == grupos_rede][['Centro_Resultado_Nome','mail','Grupos']]
        a = a.groupby(['Centro_Resultado_Nome','Grupos'],as_index=True).agg(['nunique']).reset_index()
        a = a.droplevel(1, axis=1)
        a.rename(columns={'Centro_Resultado_Nome':'target', 'Grupos':'source', 'mail':'value'}, inplace=True)        
        common_filter = a

    
    else:
        a = usuarios_localizados.loc[usuarios_localizados['Grupos'] == grupos_rede][['Cargo','mail','Grupos']]
        a = a.groupby(['Cargo','Grupos'],as_index=True).agg(['nunique']).reset_index()
        a = a.droplevel(1, axis=1)
        a.rename(columns={'Cargo':'target', 'Grupos':'source', 'mail':'value'}, inplace=True)        
        common_filter = a

        
    nodes={'start':ProcessGroup(list(common_filter['source'])),'end':ProcessGroup(list(common_filter['target'])),}
    ordering = [['start'],['end']]
    bundles = [Bundle('start','end')]
    nodes['start'].partition = Partition.Simple('source',common_filter['source'].unique())
    nodes['end'].partition = Partition.Simple('target',common_filter['target'].unique())
    sdd = SankeyDefinition(nodes, bundles, ordering)
    size_options = dict(width=700, height=800,
                 margins=dict(left=175, right=175, top=0, bottom=0))
    scale = QuantitativeScale('value', palette='Blues_3')
    b = weave(sdd,common_filter,link_color=scale).to_widget(**size_options) 
    
    with output_resultado:
        display(b)

In [ ]:

%%html
<style>
.sankey .node {
    font-size: 8pt;
}
</style>

In [ ]:
#Criando relação de filtros para ambos botões criados
def dropdown_agrupamento_eventhandler(change):
    common_filtering(change.new, dropdown_grupos_rede.value)
def dropdown_grupos_rede_eventhandler(change):
    common_filtering(dropdown_agrupamento.value, change.new)

In [ ]:

botao_iniciar_resultado = widgets.Output(layout={'border': '1px solid black'})

@botao_iniciar_resultado.capture(clear_output=True)
def iniciando(event):
    print('Você clicou no botão iniciar')
    dropdowns()
    resultado_dropdowns()
    layouts_widgets()
    return 1.0 

botao_iniciar = widgets.Button(
    description='Carregar base selecionada',
    layout={'width': '200px'}
)
botao_iniciar.on_click(iniciando)

In [ ]:
def resultado_dropdowns():
    dropdown_agrupamento.observe(dropdown_agrupamento_eventhandler, names='value')
    dropdown_grupos_rede.observe(dropdown_grupos_rede_eventhandler, names='value')
    return;

In [ ]:
def layouts_widgets():
    global item_layout, input_widgets
    item_layout = widgets.Layout(margin='0 0 100px 0')
    input_widgets = widgets.HBox([dropdown_agrupamento, dropdown_grupos_rede], layout=item_layout)
    return;

In [ ]:
def exibir_resultados():
    layouts_widgets()
    tab = widgets.Tab([output_resultado], layout=item_layout)
    tab.set_title(0, 'Sankey')
    dashboard = widgets.VBox([input_widgets, tab])
    display(dashboard)
    return;

In [ ]:
#Exibir o botão para escolha do ano    
def exibir_botao_ano_e_carregar():
    stats = interact(resposta=widgets.ToggleButtons(options=['Y','N']))
    return;

In [ ]:
exibir_botao_ano_e_carregar()
botao_iniciar

In [ ]:
#Carregando os gráficos resultantes
diagrama_venn()
categorias_n_loc()
exibir_resultados()